#### Imports

In [1]:
import pandas as pd
import datetime
import re
import requests
from requests import get
from bs4 import BeautifulSoup
import warnings
from time import sleep
from random import randint
from time import time
timestart_time = time()
warnings.warn('Warning Simulation')
from IPython.core.display import clear_output

C:\Users\k.kiki\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Warning Simulation
  if sys.path[0] == '':


In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#### Scrape official press releases and/or daily briefing transcripts and store them in a df

In [3]:
raw_html = requests.get('https://eody.gov.gr/category/deltia-typoy/')
soup = BeautifulSoup(raw_html.content, "html.parser") 
articles_container = soup.find('div',{'class':'articles-container'})
titles = articles_container.find_all('a', href=True)

# if 'νακοίνωση για την εξέλιξη της νόσου COVID-19 στη χώρα μας' in titles[0].text:
#     print(titles[0].text,titles[0].get('href'))
# else:
#     print(titles[0].text,titles[0].get('href')+', but you do not care about this announcement!!!')

frame = []

for t in titles:
    entries = {}
    
    if 'νακοίνωση για την εξέλιξη της νόσου COVID-19 στη χώρα μας' in t.text\
        or 'νημέρωση διαπιστευμένων συντακτών υγείας από' in t.text:

        entries['title'] = t.text
        entries['url'] = t.get('href')

        frame.append(entries)
    
frame = pd.DataFrame(frame)
frame['date'] = frame.title.str.extract(r'[(](\d+\/\d+\/\d+)[)]')
frame['date'] = pd.to_datetime(frame['date'])
frame['date'] = pd.to_datetime(frame["date"], format="%Y-%m-%d").dt.strftime("%Y-%m-%d")

# frame

In [4]:
def get_press_release(row):
    page = requests.get(row['url'])
    soup_doc = BeautifulSoup(page.content, "html.parser") 
    
    par = ''
    for p in soup_doc.find_all('p'):
        par = par+p.text.strip()
        
    return pd.Series({
    'content': par 
    })   

In [5]:
df = frame.apply(get_press_release, axis=1).join(frame)
# df.head()

#### Extract info about number of cases, deaths, critical cases and testing mentioned in content scraped - Clean data

In [6]:
df['new_cases'] = df.content.str.extract(r'.*[aα]νακοινώνο\w+\s+(\d+)\s+νέα\s+κρούσματα',re.IGNORECASE).astype(float)
df['total_cases'] = df.content.str.extract(r'συνολικός\s+αριθμός.*κρουσμάτων\s+είναι\s+(\d+[.]?\d+)',re.IGNORECASE)
df['total_cases'] = df.total_cases.str.replace('.','').astype(float)

In [7]:
# df['new_deaths'] = df.content.str.extract(r'(?:μεθ.*\s+)(\w+|\d+)(?:\s+ακόμα\s+καταγεγραμμένο.*θ[άα]ν[αά]το.*και)',re.IGNORECASE)
#σήμερα καταγράφουμε μηδέν θανάτους και συνολικά στη χώρα έχουμε καταγράψει
df['new_deaths'] = df.content.str.extract(r'(?:μεθ.*\w+με\s+|μεθ.*\w+με\s+ακόμα\s+)(\w+)(?:\s+καταγεγραμμένο.*θ[άα]ν[αά]το.*\s+και.*στη\s+χώρα.*|\s+ακόμα\s+καταγεγραμμένο.*θ[άα]ν[αά]το.*\s+και.*στη\s+χώρα.*|\s+νέο.*θ[άα]ν[αά]το.*και.*στη\s+χώρα.*|\s+θ[άα]ν[αά]το.*και.*στη\s+χώρα.*)',re.IGNORECASE)

In [8]:
df['new_deaths'] = df.new_deaths.replace({'μηδέν':'0',
                                          'κανέναν':'0',
                                          'κανένα':'0',
                                          "έναν": "1", 
                                          "δύο": "2", 
                                          'τρεις':'3', 
                                          'τέσσερις':'4',
                                         'πέντε':'5',
                                         'έξι':'6',
                                         'εφτά':'7',
                                         'επτά':'7',
                                         'οχτώ':'8',
                                         'οκτώ':'8',
                                         'εννέα':'9',
                                         'ενιά':'9',
                                         'δέκα':'10',
                                         'έντεκα':'11',
                                         'δώδεκα':'12'}).astype(float)

In [9]:
df['total_deaths'] = df.content.str.extract(r'(?:.*θ[άα]ν[αά]το.*και.*?\s+)(\d+).*?\s+θ[άα]ν[αά]τους.*?\s+στη\s+χώρα',re.IGNORECASE).astype(float)

In [10]:
df['intubated'] = df.content.str.extract(r'(\d+)\s+συμπολίτες\s+μας\s+νοσηλεύονται\s+διασωληνωμένοι').astype(float)

In [11]:
df['total_tests'] = df.content.str.extract(r'.*\s+ελεγχθεί\s+(\d+[.]?\d+)\s+κλινικά\s+δείγματα',re.IGNORECASE)
df['total_tests'] = df.total_tests.str.replace('.','').astype(float)

In [12]:
# ---------------- CHECKING -----------------

# df.head(1)
# df.new_cases.value_counts(dropna=False)
# df.total_cases.value_counts(dropna=False)
# df.new_deaths.value_counts(dropna=False)
# df.total_deaths.value_counts(dropna=False)
# df.intubated.value_counts(dropna=False)
# df.total_tests.value_counts(dropna=False)
# df[df.total_tests.isnull()]

# -------------------------------------------

#### Store some variables for later use

In [13]:
#current total number of cases, according to content scraped/data extracted
totalcases_s = df.total_cases[0]
#current number of new cases, according to content scraped/data extracted
newcases_s = df.new_cases[0]

#current total number of deaths, according to content scraped/data extracted
totaldeaths_s = df.total_deaths[0]
#current number of new deaths, according to content scraped/data extracted
newdeaths_s = df.new_deaths[0]

#current total number of intubated patiens, according to content scraped/data extracted
intub_s = df.intubated[0]

#current total number of tests, according to content scraped/data extracted
totaltests_s = df['total_tests'][0]

#the date of latest official announcement scraped
date_s = df['date'][0]

#today
today = pd.datetime.today().strftime("%Y-%m-%d")

#### Read our first dataset (greece.csv) and store variables for later use

In [14]:
greece = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRpR8AOJaRsB5by7H3R_GijtaY06J8srELipebO5B0jYEg9pKugT3C6Rk2RSQ5eyerQl7LolshamK27/pub?gid=1017539712&single=true&output=csv')
# ('https://raw.githubusercontent.com/iMEdD-Lab/open-data/master/COVID-19/greece.csv')

#total number of cases in our current dataset
greececasessum = greece.cases.sum()
#number of cases for which location remains unknown in our current dataset
greececasesunknown = greece[greece['county_en'] == 'Unknown']['cases']

#total number of deaths in our current dataset
greecedeadsum = greece.dead.sum()
#number of deaths for which location remains unknown in our current dataset
greecedeadunknown = greece[greece['county_en'] == 'Unknown']['dead']

#total number of intubated patients in our current dataset
greeceintubsum = greece.critical.sum()
#number of intubated patients whose region of origin remains unknown in our current dataset
greeceintubunknown = greece[greece['county_en'] == 'Unknown']['critical']

#### Compare official info announced with data in our current dataset, update our dataset if needed and notify us respectively

In [15]:
greecetext = ''

if date_s == today:

    if greececasessum < totalcases_s and greececasessum+newcases_s == totalcases_s:
        greececasesunknown = greececasesunknown+(totalcases_s-greececasessum)
        
        greece.to_csv('greece_auto_test.csv')
        greececases = 'greece.cases was updated and greece_auto_test.csv has been saved'

    elif greececasessum < totalcases_s and greececasessum+newcases_s != totalcases_s:
        greececases = 'Be careful: Something goes wrong with cases announced!!!'

    elif greececasessum >= totalcases_s:
        greececases = 'Cool: greece.cases is updated!'
        
    else:
        greececases = 'Sorry, I do not know what is happening with greece.cases!'



    if greecedeadsum < totaldeaths_s and greecedeadsum+newdeaths_s == totaldeaths_s:
        greecedeadunknown = greecedeadunknown+newdeaths_s
        
        greece.to_csv('greece_auto_test.csv')
        greecedead = 'greece.dead was updated and greece_auto_test.csv has been saved'

    elif greecedeadsum < totaldeaths_s and greecedeadsum+newdeaths_s != totaldeaths_s:
        greecedead = 'Be careful: Something goes wrong with deaths announced!!!'
        
    elif greecedeadsum >= totaldeaths_s:
        greecedead = 'Cool: greece.dead is updated!'

    else:
        greecedead = 'Sorry, I do not know what is happening with greece.dead!'


        
    if greeceintubsum != intub_s:
        greeceintubunknown = greeceintubunknown+(intub_s-greeceintubsum)
        
        greece.to_csv('greece_auto_test.csv')
        greeceintub = 'greece.critical was updated and greece_auto_test.csv has been saved'

    elif greeceintubsum == intub_s:
        greeceintub = 'Cool: greece.critical is updated!'
        
    else:
        greeceintub = 'Sorry, I do not know what is happening with greece.critical!'

    greecetext = greececases+greecedead+greeceintub
    
    
else:
    greecetext = 'Latest announcement published refers to '+str(date_s)+'. So, greece.csv should be updated.'


In [16]:
greecetext

'Latest announcement published refers to 2020-05-20. So, greece.csv should be updated.'

#### Read and manipulate our second dataset (greeceTimeline.csv)

In [17]:
greeceTimeline = pd.read_csv('https://raw.githubusercontent.com/iMEdD-Lab/open-data/master/COVID-19/greeceTimeline.csv')
# greeceTimeline = greeceTimeline.drop(greeceTimeline.columns[-1],axis=1)
grt = greeceTimeline
grt = grt.drop(["Province/State", "Country/Region"], axis=1)
grt = grt.set_index("Status")
grt = grt.T
grt = grt.reset_index()
grt = grt.rename(columns={"index": "date"})
grt['date'] = pd.to_datetime(grt['date']).dt.strftime("%Y-%m-%d")

last_date = grt['date'].iloc[-1]

grt.tail()

Status,date,cases,cruise_ship_ElVen,Ritsona,deaths,recovered,hospitalized,intensive_care,intubated,total_tests,total cases
80,2020-05-16,9.0,0.0,0.0,1.0,NaN,NaN,NaN,23.0,126283.0,2819.0
81,2020-05-17,15.0,0.0,0.0,1.0,NaN,NaN,NaN,22.0,128525.0,2834.0
82,2020-05-18,2.0,0.0,0.0,2.0,NaN,NaN,NaN,24.0,131684.0,2836.0
83,2020-05-19,4.0,0.0,0.0,1.0,NaN,NaN,NaN,22.0,136001.0,2840.0
84,2020-05-20,10.0,0.0,0.0,1.0,NaN,NaN,NaN,22.0,139445.0,2850.0


In [18]:
timeline = {}
# if grt['date'].iloc[-1] < df['date'][0]:
if last_date < today and date_s == today:

    timeline['date'] = date_s
    timeline['cases'] = newcases_s
    timeline['total cases'] = totalcases_s
    timeline['deaths'] = newdeaths_s
    timeline['intubated'] = intub_s
    timeline['total_tests'] = totaltests_s
     
    grt = grt.append(timeline,ignore_index=True)
    
    grt['date'] = grt.date.astype(str)
    grt['year'] = grt.date.str.extract(r'(\d\d)\d\d-\d\d-\d\d')
    grt['month'] = grt.date.str.extract(r'\d\d\d\d-(\d\d)-\d\d')
    grt['month'] = grt.month.replace({'01':'1',
                                   '02':'2',
                                   '03':'3',
                                   '04':'4',
                                   '05':'5',
                                   '06':'6',
                                   '07':'7',
                                   '08':'8',
                                   '09':'9'})
    grt['day'] = grt.date.str.extract(r'\d\d\d\d-\d\d-(\d\d)')
    grt['day'] = grt.day.replace({'01':'1',
                               '02':'2',
                               '03':'3',
                               '04':'4',
                               '05':'5',
                               '06':'6',
                               '07':'7',
                               '08':'8',
                               '09':'9'})
    grt['date2'] = grt.month+'/'+grt.day+'/'+grt.year
    grt = grt.drop(['date','year','month','day'],axis=1)
    
    grt = grt.set_index('date2').T.reset_index()
    grt = grt[['Status',grt.columns[-1]]].copy()
    grt_new = pd.merge(greeceTimeline,grt,on='Status',how='outer')

    grt_new.to_csv('greeceTimeline_auto_test.csv')
    
    greeceTimeline_text = 'Daily update: greeceTimeline appended and greece_auto_test.csv has been saved'

elif last_date < today and date_s != today:
    
    greeceTimeline_text = 'greece.Timeline is not updated, because no announcement has been out for today. Please wait or check out what is happening!'
    
elif last_date == today:
    
    greeceTimeline_text = 'Cool: greece.Timeline is updated!'

else: 
    greeceTimeline_text = 'Sorry, I do not know what is happening with greece.Timeline!'


#grt_new

In [19]:
greeceTimeline_text

'greece.Timeline is not updated, because no announcement has been out for today. Please wait or check out what is happening!'